In [1]:
# Download data (same as from Kaggle)
#!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"


--2023-01-12 19:19:04--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.128, 74.125.24.128, 142.250.4.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K   742KB/s    in 0.8s    

2023-01-12 19:19:11 (742 KB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [2]:
def unzip_data(filename):
    import zipfile
    """
    Unzips filename into the current working directory.

    Args:
        filename (str): a filepath to a target zip folder to be unzipped.
    """
    zip_ref = zipfile.ZipFile(filename, "r")
    zip_ref.extractall()
    zip_ref.close()
    

In [3]:
unzip_data('nlp_getting_started.zip')

In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [5]:
train_csv_path = 'train.csv'
test_csv_path = 'test.csv'
sample_solutions_path ='sample_submission.csv'

In [6]:
train_df = pd.read_csv(train_csv_path)
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [7]:
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
from sklearn.model_selection import train_test_split
train_sentence,val_sentence,train_label,val_labels = train_test_split(train_df["text"].to_numpy(),
                                                                      train_df["target"].to_numpy(),
                                                                      test_size=0.1, # dedicate 10% of samples to validation set
                                                                       random_state=42
                                                                        )

In [10]:
train_sentence.shape, train_label.shape,val_sentence.shape, val_labels.shape

((6851,), (6851,), (762,), (762,))

In [12]:
train_sentence[1], train_label[1]

('w--=-=-=-[ NEMA warns Nigerians to prepare for drought http://t.co/5uoOPhSqU3',
 1)

In [11]:

train_dataset = tf.data.Dataset.from_tensor_slices((train_sentence, train_label))
train_dataset =  train_dataset.shuffle(1341).batch(32).cache().prefetch(tf.data.AUTOTUNE)
valid_dataset = tf.data.Dataset.from_tensor_slices((val_sentence,val_labels))
valid_dataset = valid_dataset.batch(32).cache().prefetch(tf.data.AUTOTUNE)
print(f"Train : {train_dataset} \n"
      f"Test : {valid_dataset}")

Train : <PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))> 
Test : <PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


In [13]:

max_vocab = 10_000  # Maximum vocab size.
max_seq_len = 600  # Sequence length to pad the outputs to.

# Create the layer.
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=max_vocab,
    output_mode='int',
    output_sequence_length=max_seq_len)

# Now that the vocab layer has been created, call `adapt` on the
# text-only dataset to create the vocabulary. You don't have to batch,
# but for large datasets this means we're not keeping spare copies of
# the dataset.
vectorize_layer.adapt(train_sentence)

embedding_layers = tf.keras.layers.Embedding(input_dim=max_vocab,
                                     output_dim=5,
                                     embeddings_initializer="uniform",
                                     input_length = max_seq_len,
                                     name="embedding_layers")


In [14]:

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=5) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=3,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)

In [17]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = vectorize_layer(inputs)
x = embedding_layers(x)
x = layers.Conv1D(128, 5, padding = 'same', activation = 'elu')(x)
x = layers.Conv1D(64, 5, padding = 'same', activation = 'elu')(x)
x = layers.GlobalMaxPool1D()(x)
#x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(16, activation='relu')(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs, name="Conv")

model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])


Model: "Conv"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 600)              0         
 torization)                                                     
                                                                 
 embedding_layers (Embedding  (None, 600, 5)           50000     
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 600, 128)          3328      
                                                                 
 conv1d_3 (Conv1D)           (None, 600, 64)           41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0      

In [18]:
from datetime import datetime

start = datetime.now()
history = model.fit(train_dataset,
                    epochs= 10,
                    validation_data=valid_dataset,
                    verbose=1)

end = datetime.now()

print(f"The time taken to train the model is :{end - start}")
results = model.evaluate(valid_dataset)

Epoch 1/100
215/215 [==============================] - 14s 63ms/step - loss: 0.5818 - accuracy: 0.6805 - val_loss: 0.4745 - val_accuracy: 0.7913
Epoch 2/100
215/215 [==============================] - 15s 70ms/step - loss: 0.3494 - accuracy: 0.8530 - val_loss: 0.5056 - val_accuracy: 0.7822
Epoch 3/100
215/215 [==============================] - 14s 64ms/step - loss: 0.2337 - accuracy: 0.9120 - val_loss: 0.6230 - val_accuracy: 0.7388
Epoch 4/100
215/215 [==============================] - 13s 62ms/step - loss: 0.1626 - accuracy: 0.9444 - val_loss: 0.7225 - val_accuracy: 0.7428
Epoch 5/100
215/215 [==============================] - 14s 65ms/step - loss: 0.1254 - accuracy: 0.9581 - val_loss: 0.7365 - val_accuracy: 0.7559
Epoch 6/100
215/215 [==============================] - 12s 55ms/step - loss: 0.0956 - accuracy: 0.9702 - val_loss: 0.7885 - val_accuracy: 0.7612
Epoch 7/100
215/215 [==============================] - 12s 56ms/step - loss: 0.0801 - accuracy: 0.9756 - val_loss: 0.8367 - val_ac

KeyboardInterrupt: 

In [ ]:

def calculate_accuracy_results(y_true, y_pred):
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    """
     Calculates model accuracy, precision, recall and f1 score of a binary classification model.

    Args:
        y_true: true labels in the form of a 1D array
        y_pred: predicted labels in the form of a 1D array

    Returns a dictionary of accuracy, precision, recall, f1-score.
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate model precision, recall and f1 score using "weighted average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted", zero_division= 1)
    model_results = {"accuracy": model_accuracy,
                      "precision": model_precision,
                      "recall": model_recall,
                      "f1": model_f1}
    return model_results

result_preds_probs = model.predict(val_sentence)
result_preds = tf.argmax(result_preds_probs, axis=1)
val_labels_encode = tf.argmax(val_label_one_hot ,axis=1)
results = calculate_accuracy_results( 
    y_true= val_labels_encode,
    y_pred = result_preds)

print(results)

